# KNN

## Importações

In [12]:
import pandas as pd
import csv
import numpy as np
import numbers
from sklearn.preprocessing import LabelEncoder
import math
import statistics

In [13]:
# reduzir quantidade de laços de repetição
# identificar necessidade de normalização dos dados
# inserir mais métricas de desempenho (precision, recall, curva ROC, etc)

## Leitura dos dados

In [14]:
treinamento = pd.read_csv('diabetes_train.csv')

In [15]:
teste = pd.read_csv('diabetes_test.csv')

## Pré-processamento

In [16]:
class Encoders:
    def __init__(self):
        self.encoders = {}

    def add_encoder(self, feature, encoder):
        self.encoders[feature] = encoder

    def get_encoder(self, feature):
        try:
            return self.encoders[feature]
        except KeyError:
            return None

In [17]:
encoders = Encoders()

In [18]:
def cria_encoder(feature, nome, encoders):
    encoder = LabelEncoder()
    feature_codificada = encoder.fit_transform(feature)
    encoders.add_encoder(nome, encoder)
    return feature_codificada


In [19]:
def verifica_features_categoricas(dados, encoders):
    for i in range(int(dados.size/len(dados)) - 1):
        feature = dados.iloc[:, i]
        if not all(isinstance(instancia, (int, float)) for instancia in (feature.values).tolist()):
            y = encoders.get_encoder(feature.name)
            if y == None:
                dados.loc[:, feature.name] = cria_encoder((feature.values).tolist(), feature.name, encoders)
            else:
                dados.loc[:, feature.name] = y.transform((feature.values).tolist())
    
    return dados

In [20]:
treinamento = verifica_features_categoricas(treinamento, encoders)
teste = verifica_features_categoricas(teste, encoders)

In [21]:
x_treinamento, x_teste = treinamento.iloc[:, :-1].values, teste.iloc[:, :-1].values

In [22]:
x_treinamento

array([[0, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       ...,
       [2, 1, 0, ..., 0, 1, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [2, 1, 0, ..., 0, 0, 0]], shape=(312, 16), dtype=object)

In [23]:
y_treinamento, y_teste = treinamento.iloc[:, -1].values, teste.iloc[:, -1].values

## Algoritmo

In [24]:
class KNN:
    def __init__(self, k = 3):
        self.x = None
        self.y = None
        self.k = k

    def fit(self, x, y):
        self.x = x
        self.y = y

    def calcula_vizinho_mais_proximo(self, x):
        distancias = []

        for index, i in enumerate(self.x):
            d = math.sqrt(sum([math.pow(l, 2) for l in (i - x)]))
            distancias.append((d, index))

        distancias = sorted(distancias)
        menores_distancias = [distancias[i] for i in range(self.k) ]
        classes = [self.y[j] for i, j in menores_distancias]
        return statistics.mode(classes)
        

    def predict(self, x):
        y_resultados = []
        for i in x:
            y_resultados.append(self.calcula_vizinho_mais_proximo(i))

        return y_resultados

In [25]:
knn = KNN()
knn.fit(x_treinamento, y_treinamento)

In [26]:
y_resultado = knn.predict(x_teste)

In [31]:
y_teste

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1])

In [27]:
acuracia = 0
for i, k in zip(y_teste, y_resultado):
    if i == k:
        acuracia += 1
acuracia = (acuracia/len(y_teste)) * 100

In [28]:
print(f"Acurácia: {acuracia:.2f}%")

Acurácia: 96.15%


In [29]:
matriz_confusao = np.zeros((2, 2))

for i, k in zip(y_teste, y_resultado):
    matriz_confusao[i][k] += 1

In [30]:
matriz_confusao

array([[ 79.,   2.],
       [  6., 121.]])